In [3]:
!cat wordpress-deployment.yaml

apiVersion: v1
kind: Service
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  ports:
    - port: 80
  selector:
    app: wordpress
    tier: frontend
  type: LoadBalancer
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: wp-pv-claim
  labels:
    app: wordpress
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
---
apiVersion: apps/v1beta2 # for versions before 1.8.0 use apps/v1beta1
kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  selector:
    matchLabels:
      app: wordpress
      tier: frontend
  strategy:
    type: Recreate
  template:
    metadata:
      labels:
        app: wordpress
        tier: frontend
    spec:
      containers:
      - image: wordpress:4.8-apache
        name: wordpress
        env:
        - name: WORDPRESS_DB_HOST
          value: wordpress-mysql
        - name: WORDPRESS_DB_PASSWORD
          value: PASSWORDS_IN_PLAIN_TEXT_ARE_BAD_WE_WILL_SHOW_SOMETHING

In [4]:
!cat wordpress-deployment.yaml | grep PASSWORDS

          value: PASSWORDS_IN_PLAIN_TEXT_ARE_BAD_WE_WILL_SHOW_SOMETHING_MORE_SECURE_LATER


In [5]:
!kubectl create secret generic mysql-pass \
--from-literal=password=AMuchBetterWayToStoreAPassword

secret/mysql-pass created


In [6]:
!kubectl get secret

NAME                  TYPE                                  DATA      AGE
default-token-qh4bq   kubernetes.io/service-account-token   3         1d
mysql-pass            Opaque                                1         56s


In the abckground, i have changed the mysql and wordpress yaml files to include the env password using secrets. Basically i have added the following lines 

    valueFrom:
     secretKeyRef:
       name: mysql-pass
       key: password
       
in places of env variable WORDPRESS_DB_PASSWORD and MYSQL_ROOT_PASSWORD

We shall restart the services wordpress and mysql 

In [13]:
!kubectl apply -f ./mysql-deployment.yaml

service/wordpress-mysql configured
persistentvolumeclaim/mysql-pv-claim configured
error: error validating "./mysql-deployment.yaml": error validating data: ValidationError(Deployment.spec.template.spec.containers[0].env[0].value): invalid type for io.k8s.api.core.v1.EnvVar.value: got "map", expected "string"; if you choose to ignore these errors, turn validation off with --validate=false


In [14]:
!kubectl apply -f ./wordpress-deployment.yaml

service/wordpress unchanged
persistentvolumeclaim/wp-pv-claim unchanged
error: error validating "./wordpress-deployment.yaml": error validating data: ValidationError(Deployment.spec.template.spec.containers[0].env[1].value): invalid type for io.k8s.api.core.v1.EnvVar.value: got "map", expected "string"; if you choose to ignore these errors, turn validation off with --validate=false


In [16]:
!kubectl describe deployment wordpress

Error from server (NotFound): deployments.extensions "wordpress" not found


#### Some commands used in this notebook

    kubectl create secret generic db-user-pass --from-file=./username.txt     --from-file=./password.txt
    kubectl create secret generic mysql-pass --from-literal     password=PASSWORDS_IN_PLAIN_TEXT_ARE_BAD_WE_WILL_SHOW_SOMETHING_MORE_SECURE_LATER
    kubectl get secret
 
    kubectl apply -f mysql-deployment.yaml
    kubectl apply -f wordpress-deployment.yaml
    minikube service wordpress --url